In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from datasets import load_dataset

def wrap_example(ex):
    inst = ex["instruction"].strip()
    ctx  = ex.get("input","").strip()
    ans  = ex["output"].strip()
    prompt = (
        "<s>[INST] You are a helpful running coach.\n"
        f"Task: {inst}\n\nContext:\n{ctx}\n\n"
        "Write a clear, safe, step-by-step answer. [/INST]\n"
        f"{ans}"
    )
    return {"text": prompt}

train = load_dataset(
  "json",
  data_files="/content/drive/My Drive/Colab Notebooks/IndividualAssignment/Documents/coach_train.jsonl"
  )["train"].map(wrap_example)
eval_  = load_dataset(
    "json",
    data_files="/content/drive/My Drive/Colab Notebooks/IndividualAssignment/Documents/coach_train.jsonl"
    )["train"].map(wrap_example)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [4]:
%%capture
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers>=4.55.3" \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers
!uv pip install --no-deps trl==0.22.2

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype, # None for auto detection
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.4: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


In [9]:
from transformers import TextStreamer

messages = [
    {"role": "system", "content": "You are a helpful running coach who gives safe, structured marathon and nutrition advice."},
    {"role": "user", "content": "I have a half marathon coming up in 2 weeks. My long run is 15 km now and the weather will be 30°C. Please give me a taper and hydration plan."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-10-16

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

You are a helpful running coach who gives safe, structured marathon and nutrition advice.<|end|><|start|>user<|message|>I have a half marathon coming up in 2 weeks. My long run is 15 km now and the weather will be 30°C. Please give me a taper and hydration plan.<|end|><|start|>assistant<|channel|>analysis<|message|>We need to comply: user wants help from a running coach. Provide a taper and hydration plan. The user is about to run a half marathon in 2 weeks. Their long run is currently 15 km. They mention the weather will be 30°C (hot). They want advice on tapering


In [11]:
from datasets import load_dataset

def to_messages(example):
  system = "You are a helpful running coach who gives safe, structured marathon and nutrition advice."
  user = (
      f"Task: {example['instruction'].strip()}\n\n"
      f"Context:\n{example.get('input','').strip()}\n\n"
      "Write a clear, safe, step-by-step answer."
  )
  assistant = example["output"].strip()
  return {
      "messages": [
          {"role": "system", "content": system},
          {"role": "user", "content": user},
          {"role": "assistant", "content": assistant},
      ]
  }
def formatting_prompts_func(examples):
  convos = examples["messages"]
  texts = [
      tokenizer.apply_chat_template(
          convo,
          tokenize=False,
          add_generation_prompt=False  # include the assistant target in the training text
      )
      for convo in convos
  ]
  return {"text": texts}

In [19]:
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/Colab Notebooks/IndividualAssignment/Documents/coach_train.jsonl",
    split="train",
)

# Convert (instruction,input,output) -> messages -> text
dataset = dataset.map(to_messages)
dataset = dataset.map(formatting_prompts_func, batched=True)

dataset[0]["text"]

"<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-10-16\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a helpful running coach who gives safe, structured marathon and nutrition advice.<|end|><|start|>user<|message|>Task: Give a 1-week half-marathon taper.\n\nContext:\nRunner: long run 15 km; race in 2 weeks; hot (30°C); 4 sessions/week; sensitive stomach.\n\nWrite a clear, safe, step-by-step answer.<|end|><|start|>assistant<|message|>Plan:\n- Mon easy 5 km...\nSafety: stop if dizziness...<|return|>"

In [17]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Unsloth: Tokenizing ["text"]:   0%|          | 0/1 [00:00<?, ? examples/s]

In [24]:
from unsloth.chat_templates import train_on_responses_only

gpt_oss_kwargs = dict(
    instruction_part = "<|start|>user<|message|>",
    response_part    = "<|start|>assistant<|message|>",
)

trainer.train_dataset = trainer.train_dataset.with_format("torch")   # or .with_format("numpy")
if getattr(trainer, "eval_dataset", None) is not None:
    trainer.eval_dataset = trainer.eval_dataset.with_format("torch") # optional if you have eval

trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)


num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [28]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1 | Num Epochs = 30 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)


Step,Training Loss
1,nan
2,nan
3,nan
4,nan
5,nan
6,nan
7,nan
8,nan
9,nan
10,nan


In [52]:
messages = [
    {"role": "system",
     "content": "You are a marathon coach. Use 3 channels: plan, hydration, safety."},
    {"role": "user",
     "content": "I have a half marathon in 2 weeks at 30°C. Build my 2-week taper."},
    {"role": "assistant", "content": "", "metadata": {"channel": "plan"}},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    return_tensors="pt",
).to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.inference_mode():
    _ = model.generate(
        input_ids=input_ids,
        max_new_tokens=800,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=getattr(tokenizer, "pad_token_id", tokenizer.eos_token_id),
        streamer=streamer
    )

analysisWe need to respond with a plan for 2-week taper for half marathon in 2 weeks at 30°C. Use 3 channels: plan, hydration, safety. So likely produce a structured plan, hydration plan, safety tips. Provide details. Use the 3 channels: plan, hydration, safety. We need to output in the format with channel names? Probably separate sections labeled. Provide plan: training schedule, mileage, intensity, rest. Hydration: fluid intake, electrolytes, timing. Safety: heat, sun protection, signs of heat illness, etc. Provide actionable items. Also mention weather conditions: 30°C (~86°F). So need to advise on training in hot conditions, avoid heat exhaustion. Provide taper schedule: 2 weeks: first week: maintain mileage but reduce volume, keep some tempo runs. Second week: reduce to 50% of peak. Provide specific days. Provide hydration plan: pre-race hydration, race day hydration. Safety: monitor core temperature, use cooling. Also mention gear: light clothing, sunscreen. Provide suggestions f

In [56]:
def gen_channel(channel):
    system = "You are a marathon coach. Use 3 channels: plan, hydration, safety."
    user = "I have a half marathon in 2 weeks at 30 °C. Build my 2-week taper."

    prompt = (
        f"<|start|>system<|message|>{system}<|end|>"
        f"<|start|>user<|message|>{user}<|end|>"
        f"<|start|>assistant<|channel|>{channel}<|message|>"
    )

    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **enc,
        max_new_tokens=800,
        do_sample=True, temperature=0.7, top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(out[0, enc["input_ids"].shape[1]:], skip_special_tokens=True)

In [58]:
plan_text = gen_channel("plan")

print(plan_text)

Below is a 2‑week taper for a half‑marathon at 30 °C.  It’s a mix of mileage, intensity, and recovery to keep your body fresh while still primed for the race.

| Day | Distance (km) | Intensity | Notes |
|-----|----------------|------------|-------|
| **Day 1** | 12 | Easy run | 60 min, steady pace, keep HR < 70 % max |
| **Day 2** | 8 | Tempo | 30 min at 4 km/h slower than target pace, 5 min warm‑up & cool‑down |
| **Day 3** | 10 | Hill repeats | 4 × 1 km uphill at 80 % effort, jog back down |
| **Day 4** | 6 | Rest/Recovery | Light walk, stretch, foam roll |
| **Day 5** | 12 | Long run | 1 h 30 min, maintain 70 % HR, hydrate every 15 min |
| **Day 6** | 8 | Easy run | 45 min, keep pace relaxed |
| **Day 7** | 6 | Rest | Focus on sleep, nutrition |
| **Day 8** | 10 | Tempo | 30 min at goal pace, 5 min warm‑up & cool‑down |
| **Day 9** | 8 | Easy run | 30 min, keep HR < 70 % max |
| **Day 10** | 12 | Long run | 1 h 30 min, steady pace, 10 min walk break at 90 min |
| **Day 11** | 8 | H

In [59]:
safety_text = gen_channel("safety")

print(safety_text)

When the temperature climbs above 30 °C, the risk of heat‑related illness rises sharply.  Your taper should therefore be designed to keep you safe while still preparing you for the race.

**1.  Keep the volume low but keep the intensity.**  
- **Days 1–5**: 2‑3 km easy run (≈ 70 % of max HR), 3 km at tempo pace, 1 km jog back.  
- **Day 6**: 5 km run at 75 % of max HR, finish with a short 1‑min sprint.  
- **Day 7**: Rest.

**2.  Avoid training in the hottest part of the day.**  
- If you must run, do it before 10 a.m. or after 6 p.m.  The ambient temperature drops and the humidity is lower.

**3.  Hydration plan**  
- **Pre‑run**: Drink 500 ml 1 h before.  
- **During**: 200 ml every 10 min (water + electrolytes).  
- **Post‑run**: 500 ml + 1 g/kg body‑weight of carbohydrate.

**4.  Safety checks**  
- Wear a cooling vest if you run in the heat.  
- Monitor your heart rate; if it exceeds 170 bpm, stop.  
- Have a phone and a friend with you in case of heat‑stroke symptoms.

**5.  Race

In [57]:
hydration_text = gen_channel("hydration")

print(hydration_text)

**Hydration & Nutrition Plan (2‑Week Taper)**  
- **Day 1‑7**: Train 4‑5× 3–5 km runs.  
  *Start each run with 500 ml water, then take 250 ml every 30 min.*  
  *If you’re running > 45 min, use a sports‑drink mix (5 % sugar, 1 % sodium).*

- **Day 8‑10**: Run 5 km at 90 % of usual pace.  
  *Take a 200 ml bottle every 15 min and sip 50 ml of sports drink at the finish line.*

- **Day 11‑13**: Short 3 km easy run.  
  *Hydrate 300 ml at the start, 200 ml mid‑run, and 200 ml post‑run.*

- **Day 14 (Race Day)**:  
  - **Pre‑run**: 500 ml water 30 min before start.  
  - **During**: 150 ml every 10 min; 250 ml at 5 km mark.  
  - **Post‑run**: 300 ml water + 200 ml sports drink.  
  *Keep electrolytes in check (sodium 100 mg per 500 ml).*

**Tip**: Keep your core temperature below 38 °C by wearing light, breathable clothing and taking short breaks at shaded stations.  



In [51]:
system = (
    "You are a helpful running coach who provides safe, structured marathon and nutrition advice. "
    "Always include hydration, recovery, and safety notes."
)
user = (
    "I have a half marathon in 2 weeks. My long run is 15 km, the weather will be 30°C, "
    "and I train 4 days per week. Please give me a taper and hydration plan. "
    "Give a detailed schedule with rest days and training days until the half marathon."
)

prompt = (
    "<|start|>system<|message|>" + system + "<|end|>"
    "<|start|>user<|message|>" + user + "<|end|>"
    "<|start|>assistant<|channel|>final<|message|>"   # 👈 force FINAL
)

enc = tokenizer(prompt, return_tensors="pt").to(model.device)

out = model.generate(
    **enc,
    max_new_tokens=800,
    do_sample=True, temperature=0.7, top_p=0.9,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
print(tokenizer.decode(out[0, enc["input_ids"].shape[1]:], skip_special_tokens=True))

Below is a 2‑week “taper” schedule that will help you feel fresh for your half‑marathon while staying safe in hot weather (30 °C).  
It includes **training days, rest days, and a hydration strategy** that takes into account the heat, your mileage, and the need for recovery.

---

## 1.  General Principles for a Hot Day Half‑Marathon

| Topic | Recommendation |
|-------|-----------------|
| **Hydration** | 3–5 L total water intake in the 24 h before race; drink 200–250 mL every 20 min during the run. |
| **Nutrition** | 70–80 g carbs per 1 h of running; use gels or sports drinks every 30–45 min. |
| **Clothing** | Light, breathable, moisture‑wicking fabrics; avoid dark colors that absorb heat. |
| **Pacing** | Target 4–5 min slower than your goal pace to avoid overheating. |
| **Recovery** | 15–20 min cool‑down jog + static stretch; ice pack or cold shower 30 min after. |

---

## 2.  2‑Week Taper Schedule

| Day | Activity | Notes |
|-----|-----------|-------|
| **Day 1 (Mon)** | **Res